In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained mask detection model (assumes model is trained and saved)
model = load_model('mask_detection_model.h5')  # Path to your trained model

# Define labels
labels = ["Wearing Mask", "Not Wearing Mask", "Wear It Properly"]

# Start video capture
cap = cv2.VideoCapture(0)  # 0 is for the default camera

# Preprocess function (adapt as per model input requirements)
def preprocess_frame(frame):
    # Resize frame to the input size the model expects
    img = cv2.resize(frame, (224, 224))  # Resize to model input size
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    processed_img = preprocess_frame(frame)

    # Make prediction using the model
    predictions = model.predict(processed_img)
    class_index = np.argmax(predictions)  # Get the predicted class

    # Display the label on the frame
    label = labels[class_index]
    color = (0, 255, 0) if class_index == 0 else (0, 0, 255)  # Green for mask, red otherwise

    # Draw a bounding box and label on the frame
    cv2.putText(frame, label, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Show the frame with predictions
    cv2.imshow('Mask Detection', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and destroy windows
cap.release()
cv2.destroyAllWindows()
